# 二次调优

整个训练过程中,基本可以分为几步
1. 数据加载
2. 数据预处理
3. 模型加载
4. 定义LoRA参数
5. 插入微调矩阵
6. 定义训练参数
7. 构建训练器开始训练

这些流程基本是固定的,而训练调优过程中需要调整的是以下这些项:
1. 输入和输出: 数据路径, 模型路径, 输出路径
2. 参数: lora参数, 训练参数

因此,我们将整个训练过程中基本不变的部分封装到train.py

# 初始化

使用jupyter中的魔法指令 `%run` 来嵌入一个python脚本到当前的notebook

In [ ]:
%run train.py

In [ ]:
from peft import get_peft_model
from transformers import Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback
from train import load_model, build_lora_config, build_train_arguments
import os
import torch

In [ ]:
train_data_path = './datasets/train_test/test0819.jsonl'
eval_data_path = './datasets/train_test/eval0819.jsonl'
model_path = r"D:\Pretrained_models\Qwen\Qwen2-1___5B-Instruct"
output_path = './Qwen2-1___5B-Instruct_ft_0819_1'

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

加载模型和数据集

In [ ]:
model, tokenizer =  load_model(model_path, device=device)
model, tokenizer

# 调优-1
每次模型在尚未训练完所有数据时就提前结束, 可能与提前结束的配置有关, 所以先从提前结束开始调整.
- 调整点: 在构建训练器时暂时先去掉提前结束的配置,让模型跑完预设的3个epoch
- 目的: 让数据被充分的训练,避免因损失陷入局部最小值而提前结束
> 模型训练在验证损失还没有完全收敛时就提前停止的现象被成为Premature Early Stopping, 往往是因为验证损失存在短期波动, 而这个波动的成都不同的模型和数据集都不相同,一般需要观察验证损失在更长时间内的变化趋势,来合理的设置early_stopping_patience

In [ ]:
def build_trainer(model, tokenizer, train_args, lora_config, train_dataset, eval_dataset):
    peft_model = get_peft_model(model, lora_config)
    peft_model.print_trainable_parameters()
    return Trainer(
        model=peft_model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True),
        # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] #早停回调
    )

lora参数, 训练参数都先不做调整,直接复用上次的值

In [ ]:
from train import load_dataset

lora_config = build_lora_config()
train_args = build_train_arguments(output_path)
train_dataset, eval_dataset = load_dataset(train_data_path, eval_data_path, tokenizer)
trainer = build_trainer(model, tokenizer, train_args, lora_config, train_dataset, eval_dataset)
trainer.train()

小结: 在训练期间,喂给模型的数据越多,模型学到的信息就越多,让所有数据被充分的训练时前期提高模型性能的最直接途径

# 调优2
到现在为止, 小批量大小batch_size一直使用的是默认值4, 业界的经验是: `较大的batch_size有助于让梯度下降更稳定`

训练参数中,我们将batch_size调整为16, 并把梯度累计由4降到1, 总的梯度下降的batch_size其实没有改变,因为 `16x1 = 4x4` 正好借此对比两种参数设置的效果

> 注:batch_size 很消耗 gpu显存,需要找到适合自己GPU的尽可能大的值,一般是从一个小值开始,在没有报OOM的前提下逐步增大

In [ ]:
train_args = build_train_arguments(output_path)
train_args.per_device_train_batch_size=16
train_args.gradient_accumulation_steps=1

lora配置保持不变, 构建训练器开始训练

In [ ]:
lora_config = build_lora_config()
train_dataset, eval_dataset = load_dataset(train_data_path, eval_data_path, tokenizer)
trainer = build_trainer(model, tokenizer, train_args, lora_config, train_dataset, eval_dataset)
trainer.train()

执行评估测试

In [ ]:
from evaluate import evaluate
%run evaluate.py
evaluate(model, tokenizer, lora_config, train_dataset, eval_dataset)

训练小结: 梯度下降中,大的batch_size 比 小的batch_size 效果要好,总的batch_size相同的情况下,不带梯度累计要比使用梯度累积的效果好

# 调优3
在上述损失的变化数据中,可以发现: 训练后期当训练损失不断下降时, 验证损失是没有再下降,反而时有明显的上升,这至少说明可能存在两个问题
1. 模型再训练集上产生了过拟合, 训练损失在后期下降的太快
2. 模型在验证集上的泛化能力有限

chatgpt给出如下建议:
1. 增加模型的正则化力度, 对于当前lora微调场景来说,也就是lora_dropout
2. 引入学习率调度器,让学习率动态调整,特别是在训练后期,它能够让学习率逐渐减小,有助于缓解训练损失的过拟合
> 注: 使用lora进行微调时, 原始基座模型的参数是冻结不变的,那dropout操作就只能在插入的低秩矩阵上进行, 也就是lora_dropout

按照这个建议:
1. lr_scheduler_type: 学习率调度器,这里使用余弦调度器cosine,它能够在训练过程中逐渐减小学习率 有助于模型稳定
2. warmup_ratio: 学习率预热比例,0.05表示5%的steps用于预热,针对的是前面损失波动大的问题
3. 将log_level调整为info以输出一些基本的日志

In [ ]:
output_path = './Qwen2-1___5B-Instruct_ft_0213_15'
train_args = build_train_arguments(output_path)
train_args.per_device_train_batch_size=16
train_args.gradient_accumulation_steps=1
train_args.warmup_steps=0.05
train_args.lr_scheduler_type = "cosine"
train_args.log_level="info"

lora参数调整: 将lora_dropout从0.05增加到0.2,提高模型训练过程中的泛化能力

In [ ]:
lora_config = build_lora_config()
lora_config.lora_dropout=0.2

构建训练器开始训练

In [ ]:
trainer = build_trainer(model, tokenizer, train_args, lora_config, train_dataset, eval_dataset)
trainer.train()

网上有一种说法是:drop会带来模型容量的减少,可能需要同步增加lora低秩矩阵的秩r大小,以补偿模型容量的缩减
> 注: 同时调整学习率调度器和dropout并不好,无法区分两者带来的影响. 之调整lora_dropout起到了负面作用,只调整学习率调度器会起到正向作用

# 调优4
训练参数不变
lora参数部分,dropout=0.2不变, 将微调矩阵的秩r=8调整为r=16, 缩放银子lora_alpha也保持2倍的比例增加到32

In [ ]:
output_path = './Qwen2-1___5B-Instruct_ft_0213_15_28'
train_args = build_train_arguments(output_path)
train_args.per_device_train_batch_size=16
train_args.gradient_accumulation_steps=1
train_args.warmup_steps=0.05
train_args.lr_scheduler_type = "cosine"

In [1]:
lora_config = build_lora_config()
lora_config.lora_dropout=0.2
lora_config.r=16
lora_config.lora_alpha=32

NameError: name 'build_lora_config' is not defined

In [ ]:
trainer = build_trainer(model, tokenizer, train_args, lora_config, train_dataset, eval_dataset)
trainer.train()

训练小结: dropout需要和秩的大小配合调整, 增加秩的大小能够让模型学习到更多的参数,配合dropout一起调整能够提高模型的泛化能力